In [1]:
import pandas as pd 
import random
import matplotlib.pyplot as plt
import math
import numpy as np
import copy

# Homogenous case : 
p and alpha are the same for each players

In [2]:
class Individual:
    def __init__(self, initialWealth, riskCurve, alpha,index,behaviour):
        self.initialWealth = initialWealth #wealth
        self.wealth = initialWealth
        self.riskCurve = riskCurve 
        self.alpha = alpha #fraction lost during bad event
        self.behaviour = behaviour #Initialized randomly when a simulation is launched
        self.contribution = []
        self.finalWealthHistory = []#Wealth at the end of each game 
        self.index = index
        self.fitness = None
        
    def __repr__(self):
        #To output a string representaion of the object
        return ('Individual {}, behaviour : {}'.format(self.index,str(self.behaviour)))

 
    def generateRandomBehaviour(self,numberOfRounds):
        #instanciate a random behaviour
        self.behaviour = [[round(random.random(),2),round(random.random(),2),round(random.random(),2)] for i in range(numberOfRounds)]
        #self.behaviour = [[round(random.random(),2),i/numberOfRounds,i/numberOfRounds] for i in range(numberOfRounds)]
    def setBehaviour(self,behaviour):
        self.behaviour = behaviour
        
    def setFitnessValue(self):
        #to calculate the fitness value a the end of a generation (usefull to render new population)
        self.fitness =  np.exp(np.mean(self.finalWealthHistory))
        
    def Contribution(self, Cr,nround):
        beh= self.behaviour[nround]
        if(Cr<=beh[0]):
            return beh[1]
        else:
            return beh[2]
        
    
    def Step(self, Cr ,nround):
        #amount contributed to public good
        c = self.Contribution(Cr,nround)
        contribution = c*self.wealth
        #calculate what's left of wealth
        return contribution
    
    def addFinalWealthToHistory(self):
        #to add the ramaining wealth at the end of a game, the the mean is done to calculate the fitness at the end of a generation
        self.finalWealthHistory.append(self.wealth)
        
    def setWealth(self,newWealth):
        self.wealth = newWealth
        
    def resetRound(self):
        self.wealth = self.initialWealth
        
    def returnContribution(self):
        return self.contribution[-1]
        
def addNoiseToTreshold(behav,sigma):
    #add noise to the treshold in the wrightFisher process
    for elem in behav:
        elem[0] += round(np.random.normal(0,sigma),2)
        elem[0] = max(0,elem[0])
        elem[0] = min(1,elem[0])



        
def risk(Cr,riskCurve,InitialWealth,lambda1,lambda2,lambda3):#chance of having bad event happening
    
    p1 = lambda Cr :1 -(lambda1*Cr/InitialWealth)
    p2 = lambda Cr: 1-10*Cr if Cr<0.1 else 0
    p3 = lambda Cr : 1-math.pow(Cr/InitialWealth,lambda2)
    p4 = lambda Cr: 1/(math.exp(lambda3*((Cr/InitialWealth)-0.5))+1)
    dic = {'linear':p1,'piecewiseLinear':p2,'powerlaw':p3,'treshold':p4}
    if (random.random() < dic[riskCurve](Cr)):
        return 1
    return 0


In [3]:
class Simulation:
    def __init__(self,numberOfRounds,alpha,riskCurve,wealth = 1,populationSize=100,numberOfGamesByGenerations=1000,numberOfGenerations=10000,mu=0.03,sigma=0.15,catastrophicEventSchedule = 'Every Rounds',displayContribution='Total'):
        #number of round : omega in the paper
        self.numberOfRounds = numberOfRounds
        self.numberOfGamesByGenerations = numberOfGamesByGenerations
        self.numberOfGenerations = numberOfGenerations
        self.wealth = wealth 
        self.alpha = alpha
        self.riskCurve = riskCurve
        self.populationSize = populationSize
        #create the first population
        self.population = self.createRandomPopulation()
        self.mu = mu
        self.sigma = sigma
        #schedule of the catastrophic event (last round, first round, every round or final round)
        self.catastrophicEventSchedule = catastrophicEventSchedule
        #set the schedule ie the round in wich player will loose wealth
        self.schedule = self.setEventSchedule()
         #display of the contribution : total if we want the mean of cooperation for every round, Every round if we want the detail for each round
        self.displayContribution = displayContribution 
        self.lambda1 = 1
        self.lambda2 = 10
        self.lambda3 = 10
        
    def createRandomPopulation(self):
        #create the first population
        population = np.array([Individual(self.wealth,self.riskCurve,self.alpha,i,None) for i in range(self.populationSize)])
        for individual in population:
            #initialize the behaviour (lenght = numberofRounds)
            individual.generateRandomBehaviour(self.numberOfRounds)
        #set a null behaviour to help convergence
        population[0].setBehaviour([[round(random.random(),2),0,0] for i in range(self.numberOfRounds)])
        return population
    
    def setEventSchedule(self):
        #create the event schedule
        if self.catastrophicEventSchedule == 'Every Rounds':
            schedule = np.ones(self.numberOfRounds, dtype=bool)
        elif self.catastrophicEventSchedule == 'First Round':
            schedule = np.zeros(self.numberOfRounds, dtype=bool)
            schedule[0] = True
            
        elif self.catastrophicEventSchedule == 'Last Round':
            schedule = np.zeros(self.numberOfRounds, dtype=bool)
            schedule[-1] = True
            
        elif self.catastrophicEventSchedule == 'Random Round':
            schedule = np.zeros(self.numberOfRounds, dtype=bool)
            r = np.random.randint(self.numberOfRounds)
            schedule[r] = True
        else:
            raise ValueError('Incorrect shedule argument passed')
        return schedule
            
    
    def renewPopulation(self):
        """
        renew population in function of the fitness of each individual, individual with high fitness are more likely to pass
        to the next generation
        """
        fitness = np.array([individual.fitness for individual in self.population])
        fitprop = fitness/np.sum(fitness)
        numoffspring = np.random.multinomial(self.populationSize, fitprop)
        a = []
        for j in range(len(numoffspring)):
            for i in range(numoffspring[j]):
                b = [elem for elem in self.population[j].behaviour]
                #memory[counter+i] = b
                a.append(copy.deepcopy(b))
        return a
    
    def playGame(self,players):
        #play a game between two players (or more)
        Cr = 0 # initial public good 
        totalInitialWealth = np.sum([player.wealth for player in players])
        contributions=np.empty((len(players),self.numberOfRounds))
        for nround in range(self.numberOfRounds):
            #print('-----round ',nround)
            for i in range(len(players)):
                contributions[i,nround] = players[i].Step(Cr,nround)
            Cr += np.sum(contributions[:,nround])/totalInitialWealth
            
            if self.schedule[nround]:
                for i in range(len(players)):
                    newWealth = (1-self.alpha*risk(Cr,self.riskCurve,players[i].initialWealth,self.lambda1,self.lambda2,self.lambda3))*(players[i].wealth-contributions[i,nround])
                    players[i].setWealth(newWealth)
            
        for i in range(len(players)):
            players[i].contribution.append(contributions[i]/players[i].initialWealth)
                  
        for player in players:
            player.addFinalWealthToHistory()
            player.resetRound()
            
        
    def setLamda(self, lmbda):
        self.lambda1 = lmbda
        self.lambda2 = lmbda
        self.lambda3 = lmbda
        
    def playGeneration(self):
        #for a certain number of iteration 2 players are selected randomly and the play a game
        for i in range(self.numberOfGamesByGenerations):
            players = np.random.choice(self.population,2,replace=False)
            self.playGame(players)

    def wrightFisher(self): 
        #renew the population and add noise
        #print('1---------',self.population)
        newBehaviours = self.renewPopulation()
        for behav in newBehaviours:
            a = np.random.rand()
            b = np.random.rand()
            if a<self.mu:
                for elem in behav:
                    elem[0] += round(np.random.normal(0,self.sigma),2)
                    elem[0] = max(0,elem[0])
                    elem[0] = min(1,elem[0])
            if b<self.mu:
                for elem in behav:
                    elem[1] = round(np.random.uniform(0,1),2)
                    elem[2] = round(np.random.uniform(0,1),2)
        self.population = [Individual(self.wealth,self.riskCurve,self.alpha,i,newBehaviours[i]) for i in range(self.populationSize)]
        #print('2---------',self.population)
        
    def averageContribution(self):
        #Do 1 generation and return the Contributions
        contributionValues = []
        for i in range(self.numberOfGamesByGenerations):
            players = np.random.choice(self.population,2,replace=False)
            self.playGame(players)
            for p in players:
                contributionValues.append(p.contribution[-1])
        return np.mean(contributionValues, axis=0)
    
    def runSim(self):
        #lauch a simulation 
        averageResults = []

        for i in range(self.numberOfGenerations):
            self.playGeneration()
            contributionValues = []
            for j in range(self.populationSize):
                self.population[j].setFitnessValue()
                if (self.displayContribution =='Total'):
                    TotalContribution = [np.sum(elem) for elem in self.population[j].contribution]
                    mean = np.mean(TotalContribution)
                    contributionValues.append(mean)
                elif (self.displayContribution =='Every Round'):
                    mean = np.mean(self.population[j].contribution,axis = 0)
                    contributionValues.append(mean)
                else:
                    raise ValueError('incorrect dispalyContribution argument passed') 
                    
            if (self.numberOfGenerations-50<=i<self.numberOfGenerations):
                averageResults.append(np.mean(contributionValues,axis = 0))
            self.wrightFisher() 
        av = np.mean(averageResults,axis=0)
        return av


In [32]:
sim = Simulation(4,1,'linear',numberOfGenerations = 10,mu = 0.03,displayContribution = 'Every Round')
sim.runSim()

for omega = 4,alpha = 1, riskurve : linear, c = [0.55637    0.08337327 0.02496375 0.01482684]


array([0.55637   , 0.08337327, 0.02496375, 0.01482684])

In [7]:
alpha = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
riskCurve = ['piecewiseLinear','linear','treshold','powerlaw']
omega = [1,2,4]

In [100]:
results = np.zeros((len(omega),len(riskCurve),len(alpha)))
for o in omega:
    for r in riskCurve:
        for a in alpha:
            sim = Simulation(o,a,r,numberOfGenerations = 100)
            results[o,r,a] = sim.runSim() 

----------Generation 0
contribution : 0.5357
0.10314
----------Generation 0
contribution : 0.4597
0.230226
----------Generation 0
contribution : 0.4539
0.461492
----------Generation 0
contribution : 0.5138
0.61319


In [10]:
#plot Contributions for different timings of potential losses in a game with up to four rounds and four different risk curves
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
omega = [1,2,4]
riskCurve = ['piecewiseLinear','linear','treshold','powerlaw']
catastrophicEvent = ['Every Rounds','First Round','Last Round','Random Round']
i = 1
for r in riskCurve:
    for c in catastrophicEvent:
        Res = []
        for o in omega:
            sim = Simulation(o,1,r,populationSize=100,catastrophicEventSchedule = c,numberOfGenerations = 100)
            sim.runSim()
            Res.append(sim.averageContribution())
        delta = -0.2
        ax = fig.add_subplot(4, 4, i)
        for y in Res:
            N = len(y)
            x = np.array(range(N))
            ax.bar(x+delta,y,width=0.2)
            delta+=0.2
        i+=1

KeyboardInterrupt: 

In [ ]:
#plot 5:Exploring the shape of risk curves
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
i = 1
alpha = [0.1,0.3,0.5,0.7,0.9]
lmbda = [0.01,0.1,1,10,100]
riskCurve = ['linear','powerlaw','treshold']
omega = [1,4]
for o in omega:
    for r in riskCurve:
        print("omega: "+ str(o)+", Riskcurve: "+r)
        ax = fig.add_subplot(2, 3, i)
        ax.set_xscale('log')
        i+=1
        for a in alpha:
            res = []
            for l in lmbda:
                sim = Simulation(o,a,r,populationSize=20,numberOfGenerations = 20);
                sim.setLamda(l)
                sim.runSim()
                res.append(np.sum(sim.averageContribution()))
            ax.plot(lmbda,res)
            ax.set_title(str(o)+" Rounds, "+str(r) +" Riskcurve", fontsize=8)
            ax.set_ylim([0,1])


omega: 1, Riskcurve: linear


# Non Homogeneous Case


In [41]:
class Simulation:
    def __init__(self,numberOfRounds,alpha,riskCurve,wealthPoor = 1,wealthRich = 4,populationSize=100,numberOfGamesByGenerations=1000,numberOfGenerations=10000,mu=0.03,sigma=0.15,catastrophicEventSchedule = 'Every Rounds',displayContribution='Total',fractionOfPoor = 0.5):
        self.numberOfRounds = numberOfRounds
        self.fractionOfPoor = fractionOfPoor
        self.numberOfGamesByGenerations = numberOfGamesByGenerations
        self.numberOfGenerations = numberOfGenerations
        self.wealthPoor = wealthPoor
        self.wealthRich = wealthPoor
        self.alpha = alpha
        self.riskCurve = riskCurve
        self.populationSize = populationSize
        #create the first population
        self.populationPoor, self.populationRich = self.createRandomPopulation()
        self.mu = mu
        self.sigma = sigma
        self.catastrophicEventSchedule = catastrophicEventSchedule
        self.schedule = self.setEventSchedule()
        self.displayContribution = displayContribution 
        
    def createRandomPopulation(self):
        #create the first population
        population = []
        for i in len(self.populationSize):
            if np.random.rand()<self.fractionOfPoor:
                ind = Individual(self.wealthPoor,self.riskCurve,self.alpha,i,None)
                ind.generateRandomBehaviour(self.numberOfRounds)
                populationPoor.append(ind)
            else:
                ind = Individual(self.wealthRich,self.riskCurve,self.alpha,i,None)
                ind.generateRandomBehaviour(self.numberOfRounds)
                populationRich.append(ind)
                
        populationPoor[0].setBehaviour([[round(random.random(),2),0,0] for i in range(self.numberOfRounds)])
        populationRich[0].setBehaviour([[round(random.random(),2),0,0] for i in range(self.numberOfRounds)])
        return populationPoor,populationRich
    
    def setEventSchedule(self):
        if self.catastrophicEventSchedule == 'Every Rounds':
            schedule = np.ones(self.numberOfRounds, dtype=bool)
        elif self.catastrophicEventSchedule == 'First Round':
            schedule = np.zeros(self.numberOfRounds, dtype=bool)
            schedule[0] = True
            
        elif self.catastrophicEventSchedule == 'Last Round':
            self.schedule = np.zeros(self.numberOfRounds, dtype=bool)
            schedule[-1] = True
            
        elif self.catastrophicEventSchedule == 'Random Round':
            schedule = np.zeros(self.numberOfRounds, dtype=bool)
            r = np.random.randint(self.numberOfRounds)
            schedule[r] = True
        else:
            raise ValueError('Incorrect shedule argument passed')
        return schedule
            
    
    def renewPopulation(self):
        fitness = np.array([individual.fitness for individual in self.population])
        fitprop = fitness/np.sum(fitness)
        numoffspring = np.random.multinomial(self.populationSize, fitprop)
        a = []
        for j in range(len(numoffspring)):
            for i in range(numoffspring[j]):
                b = [elem for elem in self.population[j].behaviour]
                #memory[counter+i] = b
                a.append(copy.deepcopy(b))
        return a
    

            
     
        
        
    
    def playGame(self,players):
        #play a game between two players (or more)
        Cr = 0 # initial public good 
        totalInitialWealth = np.sum([player.wealth for player in players])
        contributions=np.empty((len(players),self.numberOfRounds))
        for nround in range(self.numberOfRounds):
            #print('-----round ',nround)
            for i in range(len(players)):
                contributions[i,nround] = players[i].Step(Cr,nround)
            Cr += np.sum(contributions[:,nround])/totalInitialWealth
            if self.schedule[nround]:
                for i in range(len(players)):
                    newWealth = (1-self.alpha*risk(Cr,self.riskCurve))*(players[i].wealth-contributions[i,nround])
                    players[i].setWealth(newWealth)
            
        for i in range(len(players)):
            players[i].contribution.append(contributions[i]/players[i].initialWealth)
                  
        for player in players:
            player.addFinalWealthToHistory()
            player.resetRound()
            
        
    
    def playGeneration(self):
        #for a certain number of iteration 2 players are selected randomly and the play a game
        for i in range(self.numberOfGamesByGenerations):
            population = self.populationPoor + self.populationRich
            players = np.random.choice(population,2,replace=False)
            self.playGame(players)

    def wrightFisher(self): 
        #print('1---------',self.population)
        newBehaviours = self.renewPopulation()
        for behav in newBehaviours:
            a = np.random.rand()
            b = np.random.rand()
            if a<self.mu:
                for elem in behav:
                    elem[0] += round(np.random.normal(0,self.sigma),2)
                    elem[0] = max(0,elem[0])
                    elem[0] = min(1,elem[0])
            if b<self.mu:
                for elem in behav:
                    elem[1] = round(np.random.uniform(0,1),2)
                    elem[2] = round(np.random.uniform(0,1),2)
        self.population = [Individual(self.wealth,self.riskCurve,self.alpha,i,newBehaviours[i]) for i in range(self.populationSize)]
        #print('2---------',self.population)
        
    
    def runSim(self):
        averageResults = []
        #lauch a simulation 
        for i in range(self.numberOfGenerations):
            self.playGeneration()
            contributionValuesPoor = []
            contributionValuesRich = []
            for j in range(self.populationSize):
                self.populationPoor[j].setFitnessValue()
                if (self.displayContribution =='Total'):
                    TotalContribution = [np.sum(elem) for elem in self.population[j].contribution]
                    mean = np.mean(TotalContribution)
                    contributionValues.append(mean)
                elif (self.displayContribution =='Every Round'):
                    mean = np.mean(self.population[j].contribution,axis = 0)
                    contributionValues.append(mean)
                else:
                    raise ValueError('incorrect dispalyContribution argument passed') 
                    
            if (self.numberOfGenerations-50<=i<self.numberOfGenerations):
                averageResults.append(np.mean(contributionValues,axis = 0))
            self.wrightFisher() 
        av = np.mean(averageResults,axis=0)
        print('for omega = {},alpha = {}, riskurve : {}, c = {}'.format(self.numberOfRounds,self.alpha,self.riskCurve,av))
        return av

    

            

In [43]:
a = [1,2]
b=[1,2,3]
a+b

[1, 2, 1, 2, 3]